最近、特異値分解と主成分分析について復習する機会があったので、メモ代わりに記事にしておく。

## 特異値分解と主成分分析の概要

特異値分解（SVD）と主成分分析（PCA）は推薦システムではもちろんデータ分析でも重要な手法である。

### 特異値分解（SVD）

特異値分解 (Singular Value Decomposition, SVD) は行列 $A$ を以下のように分解する手法である：

$$
A = U \Sigma V^T
$$

ここで $U$ は直交行列、$\Sigma$ は特異値の対角行列、$V$ は直交行列である。SVD は行列のランクを下げるために利用され、推薦システムでは評価行列の低ランク近似により評価予測を行う。

$$
\displaystyle R_k = U_k \Sigma_k V_k^T
$$

### 主成分分析（PCA）

主成分分析 (Principal Component Analysis, PCA) はデータの次元削減手法である。共分散行列 $C$ の固有値分解を行い、固有ベクトルを利用して次元削減を行う。PCA も SVD を用いて実装可能である。


## ソースコード

### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/linalg/03/03_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/linalg/03/03_nb.ipynb)


### 実行環境
OSはmacOSです。LinuxやUnixのコマンドとはオプションが異なりますので注意。

In [1]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [2]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行う。

In [3]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認しておきます。
ついでに乱数のseedの設定をします。

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [5]:
import random
import numpy as np

from pprint import pprint
from watermark import watermark

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


# 小数点を丸めたときに-0.0が出現するが、0.0 と -0.0 は等価であり、0.0として表示する関数
def ppprint(A):
    """
    A: np.array
        表示する行列
    """
    pprint(np.where(A == -0.0, 0, A))


print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

numpy: 1.25.2

Watermark: 2.4.3



# Pythonによる実装と具体的な説明

## 特異値分解(SVD)

特異値分解は、任意の$m \times n$行列$A$を以下のように分解する方法である：

$$
A = U \Sigma V^T
$$

ここで、$U$は$m \times m$の直交行列、$\Sigma$は$m \times n$の対角行列、$V$は$n \times n$の直交行列である。$\Sigma$の対角成分は$A$の特異値であり、非負の実数である。

### SVDの計算例

例えば、$A$が$5 \times 2$の行列であるとする：

$$
A = \begin{pmatrix}
1 & 2 \\
3 & 4 \\
7 & 6 \\
2 & 0 \\
3 & 1 \\
\end{pmatrix}
$$

この行列のSVDを求める。SVD自体はnumpyのlinalgモジュールで計算できる。以下に適当な行列$A$に対してSVDを行う実装を示す。

In [9]:
import numpy as np

A = np.array([[1, 2], [3, 4], [7, 6], [2, 0], [3, 1]])

U, S, V_T = np.linalg.svd(A, full_matrices=True)

print("=" * 40)
print("U : ")
pprint(U.round(2))

print("=" * 40)
print("S : ")
pprint(S.round(2))

# 特異値ベクトルSから特異値行列を作成
Sigma = np.zeros(A.shape)
for i in range(len(S)):
    Sigma[i, i] = S[i]

print("=" * 40)
print("Sigma : ")
pprint(Sigma.round(2))

print("=" * 40)
print("V_T : ")
pprint(V_T.round(2))

U : 
array([[-0.19, -0.37, -0.82, -0.21, -0.33],
       [-0.44, -0.45, -0.05,  0.57,  0.52],
       [-0.83,  0.06,  0.37, -0.25, -0.34],
       [-0.13,  0.59, -0.27,  0.66, -0.36],
       [-0.26,  0.55, -0.35, -0.35,  0.62]])
S : 
array([11.13,  2.24])
Sigma : 
array([[11.13,  0.  ],
       [ 0.  ,  2.24],
       [ 0.  ,  0.  ],
       [ 0.  ,  0.  ],
       [ 0.  ,  0.  ]])
V_T : 
array([[-0.75, -0.66],
       [ 0.66, -0.75]])


この結果、$U$、$S$、$V^T$が求まる。$S$は対角行列の対角成分を持つベクトルとして出力されるため、対角行列に変換する必要があることに注意する。

### SVDの利用例：推薦システム

推薦システムにおいて、行列分解を用いた方法は非常に有効である。例えば、ユーザーとアイテムの評価行列$R$をSVDにより分解することで、次元削減や潜在特徴の抽出が可能になる。これにより、ユーザーの評価傾向やアイテムの特徴を抽出し、精度の高い推薦が実現できる。

簡単にメリットとデメリットを書いておく。

#### メリット

1. 高次元データの次元削減が可能。
2. 潜在特徴の抽出により、データの本質的な構造を把握できる。

#### デメリット

1. 特に大規模データセットの場合計算コストが高い。
2. データの欠損値に対する耐性が低い。


## 主成分分析(PCA)

主成分分析は、データセットの分散を最大化するようにデータを直交座標系に変換する方法である。
PCAは、データの行列$A$に対して、次のステップで実行される。

1. データを示す行列$A$を列方向に対して平均0になるように変形し、共分散行列を計算する。

$$
\mathbf{w}_1=\underset{\mathbf{w} \neq \mathbf{0}}{\arg \max } \frac{\|\mathbf{X} \mathbf{w}\|^2}{\|\mathbf{w}\|^2} .
$$

$$
\Sigma_{i j}=\mathrm{E}\left[\left(X_i-\mu_i\right)\left(X_j-\mu_j\right)\right]=\mathrm{E}\left(X_i X_j\right)-\mathrm{E}\left(X_i\right) \mathrm{E}\left(X_j\right)
$$

$$
\mu_i=\mathrm{E}\left(X_i\right)
$$

$$
\Sigma=\mathrm{E}\left[(\mathbf{X}-\mathrm{E}[\mathbf{X}])(\mathbf{X}-\mathrm{E}[\mathbf{X}])^{\top}\right]
$$

2. 共分散行列の固有値と固有ベクトルを計算する。
3. 固有値が大きい順に固有ベクトルを選び、新しい基底を形成する。

### PCAの数式

データ行列$X$（$n \times p$行列、$n$はデータの数、$p$は特徴の数）が与えられたとき、共分散行列$C$は以下で与えられる：

$$
C = \frac{1}{n} X^T X
$$

次に、$C$の固有値$\lambda$と固有ベクトル$v$を求める。固有ベクトル$v$が主成分を表す。

### PCAの計算例

例えば、次のようなデータセットがあるとする。

$$
X = \begin{pmatrix}
2.5 & 2.4 \\
0.5 & 0.7 \\
2.2 & 2.9 \\
1.9 & 2.2 \\
3.1 & 3.0 \\
2.3 & 2.7 \\
2.0 & 1.6 \\
1.0 & 1.1 \\
1.5 & 1.6 \\
1.1 & 0.9 \\
\end{pmatrix}
$$

PythonでPCAを実装する：


In [7]:
from sklearn.decomposition import PCA
import numpy as np

X = np.array(
    [
        [2.5, 2.4],
        [0.5, 0.7],
        [2.2, 2.9],
        [1.9, 2.2],
        [3.1, 3.0],
        [2.3, 2.7],
        [2.0, 1.6],
        [1.0, 1.1],
        [1.5, 1.6],
        [1.1, 0.9],
    ]
)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

print("Principal Components:\n", principalComponents)

Principal Components:
 [[-0.82797019 -0.17511531]
 [ 1.77758033  0.14285723]
 [-0.99219749  0.38437499]
 [-0.27421042  0.13041721]
 [-1.67580142 -0.20949846]
 [-0.9129491   0.17528244]
 [ 0.09910944 -0.3498247 ]
 [ 1.14457216  0.04641726]
 [ 0.43804614  0.01776463]
 [ 1.22382056 -0.16267529]]


### PCAの利用例：推薦システム

PCAはデータの次元削減に利用され、推薦システムにおいても有効である。例えば、ユーザー評価行列の次元を削減し、計算コストを削減しつつも高い精度の推薦を実現できる。

#### メリット

1. データの次元削減により、計算コストを削減できる。
2. データの分散を最大化するため、情報の損失が少ない。
3. 視覚化が容易であり、データのパターンを把握しやすい。

#### デメリット

1. 線形変換に基づくため、非線形な関係を捉えるのが難しい。
2. 次元削減の過程で重要な情報が失われる可能性がある。

## 結論

本記事では、特異値分解と主成分分析についてPythonの実装を交えつつ解説した。

特異値分解と主成分分析は、データの次元削減や特徴抽出において非常に有効な技術である。特異値分解は推薦システムにおいて、ユーザーとアイテムの潜在特徴を抽出し、高精度な推薦を実現する。一方、主成分分析はデータの分散を最大化することで、重要な特徴を抽出し、計算コストを削減する。両者ともにメリットとデメリットが存在するため、利用目的に応じて適切な手法を選択することが重要である。

以上が特異値分解と主成分分析の概要である。これらの技術を理解し、適切に活用することで、より効果的なデータ解析や機械学習モデルの構築が可能となる。

特異値分解（SVD）の実現可能性を証明するためには、任意の $ m \times n $ 行列 $ A $ が常に次の形式に分解できることを示します：

$$
A = U \Sigma V^T
$$

ここで、$ U $ は $ m \times m $ の直交行列、$ \Sigma $ は $ m \times n $ の対角行列、$ V $ は $ n \times n $ の直交行列です。以下の手順に従って証明します。

### 手順1: 共分散行列の固有分解

まず、行列 $ A $ から始めます。

1. **共分散行列の定義**：
   $$
   A^T A
   $$
   この行列は $ n \times n $ の対称行列です。

2. **共分散行列の固有分解**：
   $$
   A^T A = V \Lambda V^T
   $$
   ここで、$ V $ は $ n \times n $ の直交行列（固有ベクトルからなる行列）、$ \Lambda $ は $ n \times n $ の対角行列（固有値を対角成分とする行列）です。

### 手順2: 右特異ベクトルと特異値

3. **右特異ベクトルの選択**：
   行列 $ A^T A $ の固有ベクトルが行列 $ V $ の列として右特異ベクトルになります。

4. **特異値の計算**：
   行列 $ \Sigma $ の対角成分（特異値）は、共分散行列 $ A^T A $ の固有値の平方根です。したがって、
   $$
   \Sigma = \text{diag}(\sqrt{\lambda_1}, \sqrt{\lambda_2}, \ldots, \sqrt{\lambda_r})
   $$
   ここで、$\lambda_i$ は $ A^T A $ の固有値です。

### 手順3: 左特異ベクトル

5. **左特異ベクトルの計算**：
   $$
   U = A V \Sigma^{-1}
   $$
   ここで、$\Sigma^{-1}$ は $\Sigma$ の逆行列であり、$\Sigma$ のゼロでない対角成分の逆数を取ります。

6. **左特異ベクトルの直交性**：
   $$
   U^T U = I
   $$
   ここで、 $ U $ が直交行列であることを確認します。実際、次のように計算できます。
   $$
   U^T U = (A V \Sigma^{-1})^T (A V \Sigma^{-1}) = \Sigma^{-1} V^T A^T A V \Sigma^{-1} = \Sigma^{-1} V^T V \Lambda V^T V \Sigma^{-1} = \Sigma^{-1} \Lambda \Sigma^{-1} = I
   $$

### 手順4: 完全なSVD

以上の手順から、行列 $ A $ は次のように分解できることがわかります。

$$
A = U \Sigma V^T
$$

これにより、任意の行列 $ A $ に対してSVDが存在することが示されました。

### 結論

この記事では、特異値分解（SVD）の実現可能性について証明しました。任意の行列 $ A $ に対して、特異値分解が存在することを示すために、共分散行列の固有分解と特異値の関係を利用しました。結果として、任意の行列は直交行列 $ U $ と $ V $、および対角行列 $ \Sigma $ によって表現できることがわかりました。

以下に特異値分解 (SVD) と主成分分析 (PCA) の違いを具体的に説明するための表を示す。

| **特徴**           | **特異値分解 (SVD)**                                                                                         | **主成分分析 (PCA)**                                                                                |
|--------------------|------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------|
| **目的**           | 行列を直交行列と対角行列に分解し、行列のランクを削減すること                                                 | データの分散を最大化する方向（主成分）を見つけ、次元削減を行う                                       |
| **適用対象**       | 任意の $m \times n$ 行列                                                                                      | 数値データの行列（データセット）                                                                     |
| **分解形式**       | $A = U \Sigma V^T$                                                                                         | 共分散行列の固有値分解                                                                               |
| **行列の種類**     | $U$：左特異ベクトルの直交行列、$\Sigma$：特異値を含む対角行列、$V^T$：右特異ベクトルの直交行列               | 固有ベクトル（主成分）と固有値（分散）                                                               |
| **出力**           | 特異ベクトル、特異値                                                                                         | 主成分、説明される分散割合                                                                            |
| **次元削減**       | 特異値の大きい部分を残し、小さい部分を削減                                                                   | 最も重要な主成分を選択し、次元削減                                                                   |
| **計算手法**       | 行列の特異値分解                                                                                             | 共分散行列の固有値分解                                                                               |
| **計算コスト**     | 高い（特に大規模データに対して）                                                                             | 比較的低い                                                                                           |
| **適用例**         | 行列補完、画像圧縮、データの低ランク近似                                                                     | データの可視化、クラスタリング前の前処理、ノイズ除去                                                 |
| **利用例**         | 推薦システムにおけるユーザー × アイテム行列の次元削減、情報検索、画像認識                                     | データ解析、機械学習の前処理、顔認識                                                                 |
| **メリット**       | 高次元データの次元削減、ノイズの除去                                                                         | データの可視化、計算効率の向上                                                                       |
| **デメリット**     | 計算コストが高い、全データが揃っている必要がある                                                             | 線形な関係しか捉えられない、データのスケーリングが必要                                                |
| **数学的基礎**     | 線形代数学                                                                                                   | 線形代数学、統計学                                                                                   |

### 具体例のPython実装比較

#### 特異値分解 (SVD) の Python 実装例

```python
import numpy as np
from pprint import pprint

# 行列 A
A = np.array([
    [3, 2],
    [2, 3],
    [1, 0]
])

# SVD
U, Sigma, VT = np.linalg.svd(A)

# 結果の表示
print("U:")
pprint(U)
print("Sigma:")
pprint(Sigma)
print("VT:")
pprint(VT)
```

#### 主成分分析 (PCA) の Python 実装例

```python
import numpy as np
from sklearn.decomposition import PCA
from pprint import pprint

# データ行列 X
X = np.array([
    [2.5, 2.4, 3.2],
    [0.5, 0.7, 1.4],
    [2.2, 2.9, 3.0],
    [1.9, 2.2, 2.8]
])

# PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

# 結果の表示
print("主成分:")
pprint(principalComponents)
print("説明される分散割合:")
pprint(pca.explained_variance_ratio_)
```

### 結論
この記事では、特異値分解 (SVD) と主成分分析 (PCA) の違いについて、表を用いて具体的に説明した。SVD は行列の特異値分解を通じて次元削減やノイズ除去に利用され、PCA はデータの分散を最大化する主成分を見つけて次元削減を行う。両者は推薦システムを含む多くの分野で重要な役割を果たしているが、それぞれの手法にはメリットとデメリットが存在する。

In [ ]:
# 主成分分析を行うPythonのコード
# ライブラリは使わないで、numpyのみで実装する


import numpy as np

# データの生成
np.random.seed(1)
X = np.random.randn(5, 3)

# データの標準化
X -= X.mean(axis=0)
X /= X.std(axis=0)

# 共分散行列の計算
n_samples = X.shape[0]
cov = np.dot(X.T, X) / n_samples

# 固有値と固有ベクトルの計算
eigenvalues, eigenvectors = np.linalg.eig(cov)

# 主成分の計算
components = np.dot(X, eigenvectors)

# 結果の表示
print("X:")
print(X)
print("共分散行列:")
print(cov)
print("固有値:")
print(eigenvalues)
print("固有ベクトル:")
print(eigenvectors)
print("主成分:")
print(components)

# 結果の確認
# 主成分分析の結果をscikit-learnで確認する
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(X)
print("scikit-learnの結果:")
print("主成分:")
print(pca.transform(X))
print("固有ベクトル:")
print(pca.components_)
print("固有値:")
print(pca.explained_variance_)
print("寄与率:")
print(pca.explained_variance_ratio_)
print("累積寄与率:")
print(np.cumsum(pca.explained_variance_ratio_))

## 結論


特異値分解と主成分分析は、データの次元削減や特徴抽出に有効な手法であり、推薦システムにおいても重要な役割を果たす。SVD による評価行列の低ランク近似や PCA による特徴抽出により、ユーザーの好みを予測し、適切なアイテムを推薦することができる。これらの手法を適用する際には、メリットとデメリットを理解し、適切な方法を選択することが重要である。